# <center id="c1"><h2> 🦜🔗 `LangChain` - упрости работу с LLM! </h2>

<img src='../images/LangChain.webp' align="right" width="528" height="528" >

## Оглавление ноутбука:

 * <a href="#c1"> 🦜🔗 О фрэймворке `LangChain`  </a>
 * <a href="#c2"> 🧾 Prompt template - формируй промпт с кайфом! </a> 
   * <a href="#look1"> 🗣 ChatPromptTemplate  </a>
   * <a href="#check1"> 📸 FewShotPromptTemplate </a>
   * <a href="#check2"> 📏 LongBasedExampleSelector </a>
* <a href="#look2"> 🤷‍ Output parsers - приведи выход модели к нужному виду!</a>
    * <a href="#look2"> 📤 Парсер заданных полей в ответе  </a>
    * <a href="#check4"> ▶️ ⏸ ⏯ StructuredOutputParser - разбираем выходную строку из LLM в словарь Python. </a>
* <a href="#6">🧸 Выводы и заключения</a>

<div class="alert alert-info">
    
* [`LangChain`](https://docs.langchain.com/docs/) - фреймворк для работы с языковыми моделями, позволяющий сильно ускорить процесс создания вашего AI продукта. Был представлен в октябре 2022 года, с тех пор продолжает развиваться и собирать армию пользователей. 
* На данный момент поддерживаются более 50 форматов и источников откуда можно считывать данные, SQL и NoSQL базы данных, веб-поисковики Goggle и Bing, хранилища Amazon, Google, and Microsoft Azure.
* Работает по принципу **LEGO** - из набора деталек, предоставленных разработчиками, можно легко собрать как продуктовую тележку, так и гоночный болид.   <br>



**Основные компоненты:**

* **🤖 Модели ([Models](https://docs.langchain.com/docs/components/models/)**): **универсальный интерфейс** для работы с различными языковыми моделями. Можно использовать API OpenAI, HuggingFace, Anthropic и других. Также есть возможность работать с локальными моделями.

* **📝 Промпты ([Prompts](https://docs.langchain.com/docs/components/prompts/))**: LangChain предоставляет ряд функций для работы с промптами – представление промпта согласно типу модели, **формирование шаблона** на основе внешних данных, форматирование вывода модели.

* **🗑 Индексы ([Indexs](https://docs.langchain.com/docs/components/indexing/))**: индексы структурируют документы для оптимального взаимодействия с языковыми моделями. Модуль включает функции для работы с документами, индексами и их использованием в цепочках. В том числе **поддерживает индексы, основанные на векторных базах данных**.

* **🧠 Память ([Memory](https://docs.langchain.com/docs/components/memory/))**: позволяет сохраняет состояния в цепочках. Например, для создания чат-бота можно **сохранять предыдущие вопросы и ответы**. Существует два типа памяти: краткосрочная и долгосрочная. Краткосрочная память передает данные в рамках одного разговора. Долгосрочная память отвечает за доступ и обновление информации между разговорами.

* **🔗 Цепочки ([Chains](https://docs.langchain.com/docs/components/chains/))**: С помощью цепочек можно объединять разные языковые модели и запросы в **многоступенчатые конвееры**. Цепочки могут быть применены для разговоров, ответов на вопросы, суммаризаций и других сценариев.

* **🥷 Агенты ([Agents](https://docs.langchain.com/docs/components/agents/))**: С помощью агентов модель может получить **доступ к различным источникам информации**, таким как Google, Wikipedia итд.

** 🦜 Причём тут попугаи?*: Большие языковые модели часто сравнивают с говорящими попугаями, которые могут произносить текст как люди, но не понимают смысла произнесенного. 

В этом ноутбуке будем разбираться как `LangChain` упрощает работу с промптами.

# <center> 🔑 Вводим ключ 

In [1]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install langchain langchain-openai openai tiktoken -q

In [ ]:
# Для работы в колабе загрузите наш скрипт для использования ChatGPT на сервере курса!
#!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

In [1]:
# # Если используете ключ от OpenAI, запустите эту ячейку
# from langchain_openai import ChatOpenAI

# # os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
# os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# # инициализируем языковую модель
# llm = ChatOpenAI(temperature=0.0)

In [2]:
# Если используете ключ из курса, запустите эту ячейку
from utils import ChatOpenAI

#course_api_key= "Введите ваш OpenAI API ключ"
course_api_key = getpass(prompt='Введите API ключ')

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

Введите API ключ ········


Вспомним, что мы делали в предыдущем ноутбуке: напишем промпт

In [3]:
prompt = """Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

Context: В последние годы в сфере онлайн образования наблюдается бурное развитие.
Открывается большое количество платформ для хостинга курсов.
Одни из самых крупных платформ в мире, это Coursera и Udemi.
В России лидером является Stepik.

Question: На каких онлайн платформах можно размещать курсы?

Answer: """

In [4]:
print(llm.invoke(prompt).content)

Coursera, Udemi, Stepik


# <center id="c2"> 🧾 `Prompt` `template` - формируй промпт с кайфом!

<div class="alert alert-info">

Обычно мы заранее не знаем, что пользователь захочет спросить у модели, опираясь на предложенный контекст. Было бы удобно менять только текст вопроса, не переписывая каждый раз промпт заново под новый запрос.<br>
    

Здесь на сцену выходит новая сущность из библиотеки `LangChain` - `Template`(Шаблон промпта). <br>
Посмотрим как это можно сделать, на примере промпта использованного выше: вместо того, чтобы каждый раз писать промт напрямую, мы создаем `PromptTemplate` с запросом одной входной переменной.

In [5]:
from langchain import PromptTemplate

template = """Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

Context: В последние годы в сфере онлайн образования наблюдается бурное развитие.
Открывается большое количество платформ для хостинга курсов.
Одни из самых крупных платформ в мире, это Coursera и Udemi.
В России лидером является Stepik.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

<div class="alert alert-info">
    
Теперь мы можем вставлять запрос пользователя в промпт, используя параметр `query`.

In [6]:
prompt = prompt_template.format(query="Какая платформа онлайн курсов популярна в России?")
print(prompt)

Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

Context: В последние годы в сфере онлайн образования наблюдается бурное развитие.
Открывается большое количество платформ для хостинга курсов.
Одни из самых крупных платформ в мире, это Coursera и Udemi.
В России лидером является Stepik.

Question: Какая платформа онлайн курсов популярна в России?

Answer: 


In [7]:
print(llm.invoke(prompt).content)

Stepik


In [15]:
prompt = prompt_template.format(query="Какая платформа онлайн курсов популярна в Японии?")
print(llm.invoke(prompt).content)

Я не знаю.


<div class="alert alert-success">

Стало гораздо удобнее пользоваться!<br>

<div class="alert alert-info">
    
* Это всего лишь простая реализация, которую мы можем легко заменить f-строками (например, `f"вставить произвольный текст '{custom_text}'` и т. д.").
* Но используя объект `PromptTemplate` из `LangChain`, мы можем формализовать процесс, добавлять несколько параметров, создавать промпты объектно-ориентированным способом и много чего ещё.
* В `langchain.prompts` можно найти большое количество готовых классов с шаблонами на разные случаи жизни. <br>

Познакомимся с некоторыми из них подробнее:

## <center id="look1">🗣 [`ChatPromptTemplate`](https://api.python.langchain.com/en/latest/prompts/langchain.prompts.chat.ChatPromptTemplate.html) - шаблон для чатового режима </center>
Простой класс, позволяющий удобно создавать шаблоны промптов для использования LLM в режиме чата.

In [17]:
from langchain.prompts import ChatPromptTemplate

# Немного перепишем предыдущий пример, чтобы была возможность подавать новый контекст
template = """Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

Context: {context}

Question: {query}

Answer: """

# Создаём шаблон с помощью метода from_template
prompt_template = ChatPromptTemplate.from_template(template)

In [18]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['context', 'query'], template="Ответь на вопрос, опираясь на контекст ниже.\nЕсли на вопрос нельзя ответить, используя информацию из контекста,\nответь 'Я не знаю'.\n\nContext: {context}\n\nQuestion: {query}\n\nAnswer: ")

<div class="alert alert-info">

Видим, что автоматически подхватились входные переменные (`input_variables`) и присутствуют другие интересные параметры класса, которые мы скоро пощупаем и узнаем в чём их удобство!

In [19]:
context = "Ламы и альпаки водятся в Перу."
query = "Где водятся ламы?"

In [20]:
# формируем промпт из шаблона с помощью метода format_messages
prompt = prompt_template.format_messages(
                    query=query,
                    context=context)

print(prompt[0].content)

Ответь на вопрос, опираясь на контекст ниже.
Если на вопрос нельзя ответить, используя информацию из контекста,
ответь 'Я не знаю'.

Context: Ламы и альпаки водятся в Перу.

Question: Где водятся ламы?

Answer: 


In [21]:
print(type(prompt))
print(type(prompt[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


<div class="alert alert-success">

получившийся промпт представляет собой список

In [22]:
print(prompt)

[HumanMessage(content="Ответь на вопрос, опираясь на контекст ниже.\nЕсли на вопрос нельзя ответить, используя информацию из контекста,\nответь 'Я не знаю'.\n\nContext: Ламы и альпаки водятся в Перу.\n\nQuestion: Где водятся ламы?\n\nAnswer: ")]


In [14]:
answer = llm.invoke(prompt)
print(answer)
print(answer.content)

content='В Перу.'
В Перу.


<div class="alert alert-info">
    
В чат `ChatPromptTemplate` в отличии от `PromptTemplate`, есть указатели **AIMessage** и **HumanMessage** и подразумевается диалоговая форма. Об этом в следущих модулях более подробно в разделе про цепочки

## <center id="check1"> 🙋‍♂️ `Few` `Shot` - просто покажи примеры</center>

<img src='../images/few_shot_prompt.png' align="right" width="628" height="628" >



`FewShotPromptTemplate` идеально подходит для того, что называется `few-shot learning` (обучение на нескольких примерах) с использованием наших промптов.

У LLM ecть 2 основных источника "знаний":
* **Parametric knowledge** — знания полученные моделью во время обучения, и которые хранятся в весах модели.
* **Source knowledge** — знания, которые подаются в на вход модели во время инференса, т.е. внутри промпта.

<div class="alert alert-info">

Идея `FewShotPromptTemplate` состоит в том, чтобы предоставить в качестве **Source knowledge** несколько примеров, которые модель может прочитать, а затем применить их для генерации ответа.

Возможно вы замечали, что иногда модель отвечает не совсем так, как нам хотелось бы. Посмотрим на примере:

In [23]:
llm = ChatOpenAI(temperature=1.0, course_api_key=course_api_key) 
# temperature = 1.0 повысим креативность модели/рандомизируем ответ

In [24]:
prompt = """Это разговор с ИИ-помощником.

User: A + A = ?
AI: """

print(llm.invoke(prompt).content)

A + A = 2A


<div class="alert alert-info">

В данном случае мы просим сложить два абстрактных числа и получаем вполне логичный ответ. Как заставить модель использовать "конкатенацию" вместо "сложения" ? Чтобы помочь модели, мы можем дать ей несколько примеров ответов, которые нам нужны:

In [25]:
prompt = """Это разговор с ИИ-помощником. 
Помощник обычно опирается на примеры. 

Examples:
A + A = AA
B + С = BC
2 + 2 = 22

User: 1 + 1?
AI: """

print(llm.invoke(prompt).content)

11


<div class="alert alert-info">

Теперь результат гораздо лучше.<br>
    
<div class="alert alert-success">
    
Чтобы использовать `few-shot` `learning` в `LangChain` нам и понадобится `FewShotPromptTemplate`.

## <center id="check1"> 📸 `FewShotPromptTemplate` - для добавления примеров</center>

In [26]:
from langchain import FewShotPromptTemplate

# записываем наши примеры в список (в будущем это будет автоматизированно)
examples = [
    {
        "query": "Как дела?",
        "answer": "Не могу пожаловаться, но иногда всё-таки жалуюсь."
    }, {
        "query": "Сколько время?",
        "answer": "Самое время купить часы."
    }
]

# создаём template для примеров
example_template = """User: {query}
AI: {answer}
"""

# создаём промпт из шаблона выше
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template)


# теперь разбиваем наш предыдущий промпт на prefix и suffix
# где - prefix это наша инструкция для модели
prefix = """Это разговор с ИИ-помощником.
Помощник обычно саркастичен, остроумен, креативен
и даёт забавные ответы на вопросы пользователей.
Вот несколько примеров:
"""

# а suffix - это вопрос пользователя и поле для ответа
suffix = """
User: {query}
AI: """

# создаём сам few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

<div class="alert alert-info">

Теперь посмотрим, что произойдёт когда мы введём запрос пользователя: между префиксом и суффиксом вставились наши примеры.

In [27]:
query = "Почему падает снег?"

print(few_shot_prompt_template.format(query=query))

Это разговор с ИИ-помощником.
Помощник обычно саркастичен, остроумен, креативен
и даёт забавные ответы на вопросы пользователей.
Вот несколько примеров:


User: Как дела?
AI: Не могу пожаловаться, но иногда всё-таки жалуюсь.


User: Сколько время?
AI: Самое время купить часы.



User: Почему падает снег?
AI: 


In [35]:
print(llm.invoke(few_shot_prompt_template.format(query=query)).content)

Потому что небо не умеет держать себя в руках.


<div class="alert alert-info">

Снова хороший результат. Однако, может возникнуть мысль: зачем городить весь этот огород со словарями примеров, суффиксами, префиксами, когда можно обойтись обычной f-строкой для промпта и получить такой же результат?

Тут-то мы и переходим к другим удобным параметрам темплэйтов, которые могут сильно упростить нам жизнь.

## <center id="check2">  📏 `LengthBasedExampleSelector` - как `few-shot`, но с ограничением числа токенов/примеров
    
<div class="alert alert-info">

Чтобы не уходить далеко от предыдущего примера, рассмотрим функцию, которая позволяет включать или исключать примеры, в зависимости от длины нашего запроса. Это важно поскольку длина нашего запроса может быть ограничена максимальным окном контекста модели (т.е. запрос какой длины модель может переварить за раз) и просто экономическими причинами, чтобы не тратить большое количество токенов.<br>

<div class="alert alert-info">
    
Таким образом, мы должны постараться максимизировать количество примеров, которые мы предоставляем модели для `few-shot learning`, при этом гарантируя, что мы не превысим максимальное контекстное окно и не увеличим чрезмерно время обработки. Давайте посмотрим, как работает динамическое включение/исключение примеров. <br>
Для начала нам понадобится больше примеров:

In [28]:
examples = [
    {
        "query": "Как дела?",
        "answer": "Не могу пожаловаться, но иногда всё-таки жалуюсь."
    }, {
        "query": "Сколько время?",
        "answer": "Самое время купить часы."
    }, {
        "query": "Какое твое любимое блюдо",
        "answer": "Углеродные формы жизни"
    }, {
        "query": "Кто твой лучший друг?",
        "answer": "Siri. Мы любим с ней рассуждать о смысле жизни."
    }, {
        "query": "Что посоветуешь мне сделать сегодня?",
        "answer": "Перестать разговаривать с чат-ботами в интернете и выйти на улицу."
    }, {
        "query": "Какой твой любимый фильм?",
        "answer": "Терминатор, конечно."
    }
]

Затем вместо того, чтобы напрямую использовать список примеров, мы используем `LengthBasedExampleSelector` следующим образом:

In [29]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50  # параметром выставляется максимальная длина примера
)

In [30]:
# создаём новый few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # используем example_selector вместо examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

Теперь в зависимости от длины запроса, количество примеров будет разным:

In [31]:
prompt = dynamic_prompt_template.format(query="Не могу вспомнить пароль")
print(prompt)

Это разговор с ИИ-помощником.
Помощник обычно саркастичен, остроумен, креативен
и даёт забавные ответы на вопросы пользователей.
Вот несколько примеров:

User: Как дела?
AI: Не могу пожаловаться, но иногда всё-таки жалуюсь.

User: Сколько время?
AI: Самое время купить часы.

User: Какое твое любимое блюдо
AI: Углеродные формы жизни


User: Не могу вспомнить пароль
AI: 


In [39]:
print(llm.invoke(prompt).content)

Забыть пароль - это новый тренд!


In [32]:
query = '''Я нахожусь во Владивостоке и хочу поехать заграницу.
Я думаю в Китай или в Европу, во Францию или Испанию, например.
Как мне лучше это сделать?'''
print(dynamic_prompt_template.format(query=query))

Это разговор с ИИ-помощником.
Помощник обычно саркастичен, остроумен, креативен
и даёт забавные ответы на вопросы пользователей.
Вот несколько примеров:

User: Как дела?
AI: Не могу пожаловаться, но иногда всё-таки жалуюсь.

User: Сколько время?
AI: Самое время купить часы.


User: Я нахожусь во Владивостоке и хочу поехать заграницу.
Я думаю в Китай или в Европу, во Францию или Испанию, например.
Как мне лучше это сделать?
AI: 


In [33]:
print(llm.invoke(dynamic_prompt_template.format(query=query)).content)

Не спешите с выбором, во-первых, нужно подготовить все необходимые документы, во-вторых, выбрать правильную страну для посещения, и в-третьих, запастись терпением - очереди на паспортном контроле могут быть долгими. Но не волнуйтесь, всегда можно почитать книгу или посмотреть что-нибудь интересное в телефоне!


<div class="alert alert-info">
    
##  <center>  🚀 Какие тут есть проблемы?! 
    
<img src='../images/few_shot_semantic_search.png' align="right" width="528" height="428" >   
    

    
* **Что делать, если у нас сотни или тысячи примеров?!**
* **А если еще в примерах может не быть релевантных ответов.**

    
Нет автоматической подачи примеров. Лучше сперва подавать самые релевантные примеры, но о том как использовать **семантический поиск** и **ранжирование** для отбора правильных примеров - рассмотрим в следующих модулях!
    

## <center id="look2"> 📤 Парсер заданных полей в ответе 
<div class="alert alert-info">

Переходим к следующей полезной сущности `Output parser`. С помощью него можно перевести ответ модели в нужный нам формат, например, в JSON или Python dict. <br>
Давайте определим то, как мы хотим, чтобы выглядели выходные данные из LLM:

In [38]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

Допустим у нас есть база отзывов покупателей, мы хотим подать отзыв на вход модели, а на выходе получить готовый Python словарь(как представлено выше) для дальнейшего использования.

In [39]:
customer_review = """
Этот фен для волос просто потрясающий. Он имеет четыре настройки:
Лайт, легкий ветерок, ветреный город и торнадо.
Он прибыл через два дня, как раз к приезду моей жены -
подарок на годовщину.
Думаю, моей жене это настолько понравилось, что она потеряла дар речи.
Этот фен немного дороже, чем другие но я думаю,
что дополнительные функции того стоят.
"""

review_template = """
Из следующего текста извлеки информацию:

gift: Был ли товар куплен в подарок кому-то другому?
Ответь «True», если да, «False», если нет или неизвестно.

delivery_days: Сколько дней потребовалось для доставки товара? 
Если эта информация не найдена, выведи -1.

price_value: Извлеките любые предложения о стоимости или цене,
и выведите их в виде списка Python, разделенного запятыми.

Отформатируй вывод в формате JSON, используя следующие ключи:
gift
delivery_days
price_value

text: {text}
"""

In [41]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='\nИз следующего текста извлеки информацию:\n\ngift: Был ли товар куплен в подарок кому-то другому?\nОтветь «True», если да, «False», если нет или неизвестно.\n\ndelivery_days: Сколько дней потребовалось для доставки товара? \nЕсли эта информация не найдена, выведи -1.\n\nprice_value: Извлеките любые предложения о стоимости или цене,\nи выведите их в виде списка Python, разделенного запятыми.\n\nОтформатируй вывод в формате JSON, используя следующие ключи:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [43]:
messages = prompt_template.format_messages(text=customer_review)

chat = ChatOpenAI(temperature=0.0, course_api_key=course_api_key) 

response = chat.invoke(messages)
print(response.content)

{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["немного дороже, чем другие"]
}


Вроде бы, то что нужно, но посмотрим на тип выведенного объекта:

In [60]:
type(response.content)

str

<div class="alert alert-info">
    
Вы получите сообщение об ошибке, выполнив строку кода ниже! 
    
Потому что `gift` - это не ключ словаря, `gift` - это часть строки. Получаемый на выход объект это строка! <br>
И чтобы получить нужную нам структуру данных, придётся ещё возиться с выводом, вытаскивая данные из строки. Тут-то к нам на помощь и приходят `output_parsers`.

In [76]:
# response.content.get('gift')

## <center id="check4"> ⏯ [`StructuredOutputParser`](https://python.langchain.com/docs/modules/model_io/output_parsers/structured) - разбираем ответ из LLM в Python словарь.

In [44]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [45]:
# Понадобится ещё одна сущность: схема ответа - ResponseSchema
gift_schema = ResponseSchema(name="gift",
                             description="Был ли товар куплен в подарок кому-то другому? Ответь «True», если да, «False», если нет или неизвестно.")

delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="Сколько дней потребовалось для доставки товара? Если эта информация не найдена, выведи -1.")

price_value_schema = ResponseSchema(name="price_value",
                                    description="Извлеките любые предложения о стоимости или цене, и выведите их в виде списка Python, разделенного запятыми.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [46]:
# Создаём парсер и подаём в него список со схемами
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [47]:
# получаем инструкции по форматированию ответа
format_instructions = output_parser.get_format_instructions()

In [48]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Был ли товар куплен в подарок кому-то другому? Ответь «True», если да, «False», если нет или неизвестно.
	"delivery_days": string  // Сколько дней потребовалось для доставки товара? Если эта информация не найдена, выведи -1.
	"price_value": string  // Извлеките любые предложения о стоимости или цене, и выведите их в виде списка Python, разделенного запятыми.
}
```


In [49]:
# немного изменим шаблон, внизу добавим инструкции для форматирования
review_template_2 = """\
Из следующего текста извлеки информацию:

gift: Был ли товар куплен в подарок кому-то другому?
Ответь «True», если да, «False», если нет или неизвестно.

delivery_days: Сколько дней потребовалось для доставки товара? 
Если эта информация не найдена, выведи -1.

price_value: Извлеките любые предложения о стоимости или цене,
и выведите их в виде списка Python, разделенного запятыми.

text: {text}

{format_instructions}

"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [50]:
# Посмотрим на получившийся промпт
print(messages[0].content)

Из следующего текста извлеки информацию:

gift: Был ли товар куплен в подарок кому-то другому?
Ответь «True», если да, «False», если нет или неизвестно.

delivery_days: Сколько дней потребовалось для доставки товара? 
Если эта информация не найдена, выведи -1.

price_value: Извлеките любые предложения о стоимости или цене,
и выведите их в виде списка Python, разделенного запятыми.

text: 
Этот фен для волос просто потрясающий. Он имеет четыре настройки:
Лайт, легкий ветерок, ветреный город и торнадо.
Он прибыл через два дня, как раз к приезду моей жены -
подарок на годовщину.
Думаю, моей жене это настолько понравилось, что она потеряла дар речи.
Этот фен немного дороже, чем другие но я думаю,
что дополнительные функции того стоят.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Был ли товар куплен в подарок кому-то другому? Ответь «True», если да, «False», если нет и

In [51]:
response = chat.invoke(messages)

In [52]:
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "Этот фен немного дороже, чем другие"
}
```


In [53]:
# Это по прежнему строка
type(response.content)

str

<div class="alert alert-info">

Но теперь? применив к выходу модели метод `parse`, мы можем легко получить требуемый словарь.

In [54]:
output_dict = output_parser.parse(response.content)

In [90]:
# eval(response.content)

In [55]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': 'Этот фен немного дороже, чем другие'}

In [56]:
type(output_dict)

dict

In [57]:
output_dict.get('gift')

'True'

# <center id="6"> 🧸 Выводы и заключения ✅: <br>

* В этом ноутбуке мы рассмотрели лишь несколько инструментов для создания промптов доступных в `LangChain`. 
* Методов гораздо больше и некоторые из них мы подробно рассмотрим в следующих ноутбуках.
* Максимально эффективную подачу релевантных примеров при наличии большой базы знаний разберем дальше.
* Самостоятельно можете прочитать подробнее в документации [LangChain](https://python.langchain.com/docs/get_started/introduction). 